In [24]:
import numpy as np
import pandas as pd
import scipy
import sklearn
from sklearn.manifold import TSNE
import pickle
# from Bio.Cluster import kcluster
import os
import numpy as np
import yaml
import os
import pandas
from collections import Counter
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import sys
import seaborn as sns
# import pyclustering
import matplotlib.pylab as plt
import seaborn as sb
from scipy.spatial.distance import cdist

from shutil import copyfile
import matplotlib as plt
# ------- SERVER EXTENSIONS ---------
lib =  r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities/droplet_dataset'
lib2 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities'
lib3 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/data_analysis'
lib4 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy'
lib5 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/scripts'
import sys
import pandas as pd, os
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)
sys.path.append(lib5)
# ------- SERVER EXTENSIONS ---------

# Creating FASTQ files
* 2 samples each time. you should only specify their names and the path for the joint CSV.
* Should be a folder with CSV mapping the barcodes for both samples.
* The output is a bash file which you can run and create the complete FASTQ files.

In [2]:
path = r'/storage/md_keren/shitay/10X/M159_M160/M159_M160_10-23-20.csv' ## csv path

sample1_name = 'M159'
sample2_name = 'M160'

output_bash_file = fr'/storage/md_keren/shitay/10X/M159_M160/pkgs/SN0219751/combine_fastq_{sample1_name}_{sample2_name}.sh' ## the path of the created .sh file
FASTQ_output_path = r'/storage/md_keren/shitay/10X/fastq' ## the output path of the final FASTQ file after you run the .sh file that build the fastq


# It is a unique prefix that each of the row files contains. 
# look over the folder '/storage/md_keren/shitay/10X/M149_M150/pkgs/SN0217504/',
# the prefix there for example is '_HVM2VBGXG.' 
# prefix = '_HVM2VBGXG.' 
prefix = '_HVWLVBGXH.'

In [3]:
df = pd.read_csv(path)[['SampleName', 'IndexBarcode1']]
values = df.values 
sample1_values = [[s[0],s[1]] for s in values if sample1_name in s[0]]
sample1_values = {int(barcode[0].split('_')[1]): barcode[1] for barcode in sample1_values}
sample2_values = [[s[0],s[1]] for s in values if sample2_name in s[0]]
sample2_values = {int(barcode[0].split('_')[1]): barcode[1] for barcode in sample2_values}


y = [[1, 2],[3, 4]]
[z for x in y for z in x]
sample1_values

{1: 'AAATGTGC', 2: 'GGGCAAAT', 3: 'TCTATCCG', 4: 'CTCGCGTA'}

In [4]:

def synthesize_command_for_sample(barcode_values, sample_name):
    n_of_channels = 4
    n_of_directions = 2
    n_of_barcodes = len(barcode_values)
    lines = []
    lines.append(f'mkdir {FASTQ_output_path}/{sample_name}')
    for direction in range(1, 3):
        line = 'cat '
        for channel in range(1, n_of_channels+1):
            for barcode_idx in range(1, n_of_barcodes+1):
                barcode = barcode_values[barcode_idx]
                line += str(channel)+prefix+str(channel)+'.'+barcode+'.unmapped.'+str(direction)+ '.fastq.gz '
        line += f'> {FASTQ_output_path}/{sample_name}/{sample_name}_S1_L001_R{direction}_001.fastq.gz'
        lines.append(line)      
    return lines
res = synthesize_command_for_sample(sample1_values, sample1_name) + synthesize_command_for_sample(sample2_values, sample2_name)
res

['mkdir /storage/md_keren/shitay/10X/fastq/M159',
 'cat 1_HVWLVBGXH.1.AAATGTGC.unmapped.1.fastq.gz 1_HVWLVBGXH.1.GGGCAAAT.unmapped.1.fastq.gz 1_HVWLVBGXH.1.TCTATCCG.unmapped.1.fastq.gz 1_HVWLVBGXH.1.CTCGCGTA.unmapped.1.fastq.gz 2_HVWLVBGXH.2.AAATGTGC.unmapped.1.fastq.gz 2_HVWLVBGXH.2.GGGCAAAT.unmapped.1.fastq.gz 2_HVWLVBGXH.2.TCTATCCG.unmapped.1.fastq.gz 2_HVWLVBGXH.2.CTCGCGTA.unmapped.1.fastq.gz 3_HVWLVBGXH.3.AAATGTGC.unmapped.1.fastq.gz 3_HVWLVBGXH.3.GGGCAAAT.unmapped.1.fastq.gz 3_HVWLVBGXH.3.TCTATCCG.unmapped.1.fastq.gz 3_HVWLVBGXH.3.CTCGCGTA.unmapped.1.fastq.gz 4_HVWLVBGXH.4.AAATGTGC.unmapped.1.fastq.gz 4_HVWLVBGXH.4.GGGCAAAT.unmapped.1.fastq.gz 4_HVWLVBGXH.4.TCTATCCG.unmapped.1.fastq.gz 4_HVWLVBGXH.4.CTCGCGTA.unmapped.1.fastq.gz > /storage/md_keren/shitay/10X/fastq/M159/M159_S1_L001_R1_001.fastq.gz',
 'cat 1_HVWLVBGXH.1.AAATGTGC.unmapped.2.fastq.gz 1_HVWLVBGXH.1.GGGCAAAT.unmapped.2.fastq.gz 1_HVWLVBGXH.1.TCTATCCG.unmapped.2.fastq.gz 1_HVWLVBGXH.1.CTCGCGTA.unmapped.2.fastq.gz 2_HVW

In [5]:
# output_bash_file
with open(output_bash_file, "w") as text_file:
    for line in res:
        text_file.write(line)
        text_file.write('\n')

In [23]:
def create_submit_file(sample_name):
    submit_content = f'executable   = run_{sample_name}.sh\n'
    submit_content += 'arguments    = $(Process)\n'

    submit_content += f'error   = /storage/md_keren/shitay/outputs/jobs/errors/err_{sample_name}\n'
    submit_content += f'output  = /storage/md_keren/shitay/outputs/jobs/out/out_{sample_name}\n'
    submit_content += f'log     = /storage/md_keren/shitay/outputs/jobs/logs/log_{sample_name}.log\n'

    submit_content += 'request_cpus   = 10\n'
    submit_content += 'request_memory = 32000\n'

    submit_content += 'should_transfer_files = no\n'
    submit_content += 'queue 1'
    
    
    path = fr'/srv01/technion/shitay/scripts/FASTQ_scripts/submit_{sample_name}.submit'
    with open(path, 'w') as file:
        file.write(submit_content)
    
create_submit_file(sample1_name)
create_submit_file(sample2_name)

In [22]:
def create_bash_file(sample_name):

    sh_content = '#!/bin/bash\n'

    sh_content += f'CELLRANGER_PATH=/Local/md_keren/cellranger-3.1.0/\n'
    sh_content += f'TRANSCRIPTOME_PATH=/storage/md_keren/CellRangerRefData/refdata-cellranger-hg19-3.0.0\n'
    sh_content += f'FASTQ_PATH=/storage/md_keren/shitay/10X/fastq/{sample_name}/\n'
    sh_content += f'ID={sample_name}\n'
    sh_content += f'SAMPLE={sample_name}\n'
    sh_content += f'MEMPERCORE=10\n'
    sh_content += f'mkdir /storage/`id -gn`/`id -un`/jobs/{sample_name}\n'
    sh_content += f'cd /storage/`id -gn`/`id -un`/jobs/{sample_name}\n'
    sh_content += '${CELLRANGER_PATH}/cellranger count --id=${ID} --sample=${SAMPLE} --fastqs=${FASTQ_PATH} --localmem=32 --localcores=8 --jobmode=local --transcriptome=${TRANSCRIPTOME_PATH} --chemistry=auto'
    
    path = fr'/srv01/technion/shitay/scripts/FASTQ_scripts/run_{sample_name}.sh'
    with open(path, 'w') as file:
        file.write(sh_content)
    
create_bash_file(sample1_name)
create_bash_file(sample2_name)